In [1]:
from datasets import load_dataset
import json

import openai
import csv

In [2]:
dataset = load_dataset("umarbutler/open-australian-legal-qa", trust_remote_code=True)

KeyError: 'tags'

In [ ]:
def pretty_print(data):
    print(json.dumps(data, indent=4, ensure_ascii=False))

In [ ]:
sample_data = dataset["train"][:3]
for idx, entry in enumerate(sample_data):
    print(f"Sample {idx + 1}:")
    pretty_print(entry)
    print("\n" + "=" * 50 + "\n")

In [ ]:
print(dataset["train"][0])

In [ ]:
def load_api_key(file_path):
    with open(file_path, 'r') as file:
        api_key = file.read().strip()
    return api_key

api_key = load_api_key('api_key.txt')

In [ ]:
# Define functions for input & output

# For getting the chat GPT output
def get_completion(prompt, model="gpt-4", temperature=0.7, max_tokens=5000, top_p=1.0, frequency_penalty=0, presence_penalty=0):
  try:
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model = model,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p = top_p,
        frequency_penalty = frequency_penalty,
        presence_penalty = presence_penalty
    )
    if response.choices:
      return response
    else:
      return "No response from the model."
  except Exception as e:
    return f"Error: {str(e)}"

def get_answer(response):
    return response.choices[0].message.content

# For loading the input text and formatting it
def load_file(file_path):
    with open(file_path, mode='r') as file:
        csv_reader = csv.reader(file)
        next(csv_reader) # skip header
        rows_lst = list(csv_reader)
    return rows_lst

def lst2str(lst):
    res_str = ""
    for i in lst:
        res_str += ','.join(i) + '\n'
    return res_str

In [ ]:
def build_dataset(tokenizer, dataset_name, input_min_text_length=2, input_max_text_length=8):
  #system_input = "Answer the following question in one sentence:"
  #prompt = f"### System:\n{system_input}\n### User:\n{question}\n### Assistant:\n"
  EOS_TOKEN = tokenizer.eos_token
  train_dataset = load_dataset(dataset_name)
  num_proc = 4
  def preprocess_function(examples):
        new_examples = {
            "query": [],
            "input_ids": [],
        }
        for question in examples["question"]:
            system_input = "Answer the following question in one sentence:"
            prompt = f"### System:\n{system_input}\n### User:\n{question}\n### Assistant:\n"
            query = prompt.format(system_input, question) + EOS_TOKEN
            tokenized_question = tokenizer(query, truncation=True)
            new_examples["query"].append(query)
            new_examples["input_ids"].append(tokenized_question["input_ids"])

        return new_examples
  ds = train_dataset.map(
          preprocess_function,
          batched=True,
          num_proc=num_proc,
          #remove_columns=original_columns,
      )
  ds = ds.filter(lambda x: len(x["input_ids"]) < 2048, batched=False)
  ds.set_format(type="torch")
  return ds

In [ ]:
def collator(data):
  return dict((key, [d[key] for d in data]) for key in data[0])

In [ ]:
dataset = build_dataset(tokenizer, "elsayedissa/alignment-questions")
dataset['train'][0]